In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.3 MB/s eta 0:00:00


In [ ]:
!pip install mne-bids


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.6/163.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
import csv
import pandas as pd
import mne
from mne_bids import (
    BIDSPath,
    print_dir_tree,
    find_matching_paths,
    get_entity_vals,
)
import numpy as np
from mne.time_frequency import psd_array_welch
import sys


In [ ]:
######################################################################################################
def calculate_relative_band_power(psds, freqs, bands):
    band_powers = {}
    total_power = np.sum(psds, axis=-1, keepdims=True)
    for band, (fmin, fmax) in bands.items():
        idx_band = np.logical_and(freqs >= fmin, freqs <= fmax)
        band_power = np.sum(psds[:,:, idx_band], axis=-1, keepdims=True)
        band_powers[band] = band_power / total_power
    return np.concatenate([band_powers[band] for band in bands], axis=-1)

####################################################################################################################

# Paths to my Google Drive where BIDS Root and CSV files are located
drive_bids_root = '/content/drive/MyDrive/free_topic /BIDS'
drive_output_dir = '/content/drive/MyDrive/free_topic /eeg_results'

# specify the BIDS path
print_dir_tree(drive_bids_root, max_depth=5)

# read the BIDS dataset
entity_key = 'subject'
sessions = get_entity_vals(drive_bids_root, entity_key)

datatype = 'eeg'
extensions = [".set"]

bids_paths = find_matching_paths(drive_bids_root, datatypes=datatype, extensions=extensions)

# Group the data
participants_tsv = os.path.join(drive_bids_root, 'participants.tsv')
participants_df = pd.read_csv(participants_tsv, sep='\t')
grouped = participants_df.groupby('Group')
bids_paths_grouped = {'A': [], 'F': [], 'C': []}

# Populate the BIDS paths lists for each group
for group_name, group_df in grouped:
    if group_name in bids_paths_grouped:
        for _, row in group_df.iterrows():
            subject = row['participant_id'].replace('sub-', '')
            bids_path = BIDSPath(subject=subject, datatype='eeg', root=drive_bids_root)

            task_path = bids_path.copy()               # Create a copy of bids_path
            task_path.update(task='eyesclosed')

            if os.path.exists(task_path.fpath):       # Check if the task path exists
                # Initialize list if not present
                if group_name not in bids_paths_grouped:
                    bids_paths_grouped[group_name] = []
                bids_paths_grouped[group_name].append(task_path)
            else:
                print(f'File not found for subject {subject} in group {group_name}')

bands = {
    'Delta': (0.1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 13),
    'Beta': (13, 30),
    'Gamma': (30, 40)
}

class suppress_output:
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr

# CSV Columns
columns = ["subject_id", "label", "band_data"]
data_list = []

# Process each group
for group_name, paths in bids_paths_grouped.items():
    print(f"\nProcessing Group {group_name}...")

    for i, path in enumerate(paths):
        with suppress_output():
            participant_id = path.entities['subject']

            # Load raw data
            raw = mne.io.read_raw_eeglab(path.fpath, preload=True, verbose=0)
            raw.pick_types(eeg=True)

            # Create fixed-length epochs
            epochs = mne.make_fixed_length_epochs(raw, duration=4, overlap=2, preload=True, verbose=0)

            # Compute PSD using psd_array_welch
            sfreq = raw.info['sfreq']
            psds = []
            freqs = None
            for epoch in epochs.get_data():
                psd, freqs = psd_array_welch(epoch, sfreq=sfreq, fmin=0, fmax=45, n_fft=1024, verbose=0)
                psds.append(psd)
            psds = np.array(psds)

            # Calculate Relative Band Power
            rel_band_power = calculate_relative_band_power(psds, freqs, bands)

            for j in range(rel_band_power.shape[0]):
                band_data = rel_band_power[j].tolist()  # Convert to list
                data_list.append([participant_id, group_name, band_data])


df = pd.DataFrame(data_list, columns=columns)
csv_file_path = os.path.join(drive_output_dir, 'eeg_results.csv')
df.to_csv(csv_file_path, index=False, quoting=csv.QUOTE_MINIMAL)

print(f"Results saved to {csv_file_path}")


|BIDS/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- task-FaceRecognition_events.json
|--- code/
|------ addHEDTags.m
|------ dInfo.m
|------ fMap.mat
|------ wakeman-henson-tags.tsv
|------ wh_bids_export_eeglab.m
|------ wh_extracteeg_BIDS.m
|--- derivatives/
|------ sub-001/
|--------- eeg/
|------------ sub-001_task-eyesclosed_eeg.set
|------ sub-002/
|--------- eeg/
|------------ sub-002_task-eyesclosed_eeg.set
|------ sub-003/
|--------- eeg/
|------------ sub-003_task-eyesclosed_eeg.set
|------ sub-004/
|--------- eeg/
|------------ sub-004_task-eyesclosed_eeg.set
|------ sub-005/
|--------- eeg/
|------------ sub-005_task-eyesclosed_eeg.set
|------ sub-006/
|--------- eeg/
|------------ sub-006_task-eyesclosed_eeg.set
|------ sub-007/
|--------- eeg/
|------------ sub-007_task-eyesclosed_eeg.set
|------ sub-008/
|--------- eeg/
|------------ sub-008_task-eyesclosed_eeg.set
|------ sub-009/
|--------- eeg/
|------------ 